In [28]:
import pandas as pd
customer_df=pd.read_csv('customer_data1.csv')
customer_df

,invoice_no,customer_id,gender,age,category,quantity,price,payment_method,invoice_date,shopping_mall
0,I138884,C241288,Female,28,Clothing,5,1500,Credit Card,5/8/2022,Kanyon
1,I317333,C111565,Male,21,Shoes,3,1801,Debit Card,12/12/2021,Forum Istanbul
2,I127801,C266599,Male,20,Clothing,1,300,Cash,9/11/2021,Metrocity
3,I173702,C988172,Female,66,Shoes,5,3001,Credit Card,16/05/2021,Metropol AVM
4,I337046,C189076,Female,53,Books,4,61,Cash,24/10/2021,Kanyon
...,...,...,...,...,...,...,...,...,...,...
99452,I219422,C441542,Female,45,Souvenir,5,59,Credit Card,21/09/2022,Kanyon
99453,I325143,C569580,Male,27,Food & Beverage,2,10,Cash,22/09/2021,Forum Istanbul
99454,I824010,C103292,Male,63,Food & Beverage,2,10,Debit Card,28/03/2021,Metrocity
99455,I702964,C800631,Male,56,Technology,4,4200,Cash,16/03/2021,Istinye Park


In [29]:
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99457 entries, 0 to 99456
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   invoice_no      99457 non-null  object
 1   customer_id     99457 non-null  object
 2   gender          99457 non-null  object
 3   age             99457 non-null  int64 
 4   category        99457 non-null  object
 5   quantity        99457 non-null  int64 
 6   price           99457 non-null  int64 
 7   payment_method  99457 non-null  object
 8   invoice_date    99457 non-null  object
 9   shopping_mall   99457 non-null  object
dtypes: int64(3), object(7)
memory usage: 7.6+ MB


Changing dateformat from object to datetime

In [30]:
customer_df['invoice_date']=pd.to_datetime(customer_df['invoice_date'], format="%d/%m/%Y")
customer_df['invoice_date'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 99457 entries, 0 to 99456
Series name: invoice_date
Non-Null Count  Dtype         
--------------  -----         
99457 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 777.1 KB


Calculation of Recency

In [31]:
from datetime import datetime
current_time=datetime.now()
customer_df['datediff']=current_time-customer_df['invoice_date']
customer_df['datediff']=customer_df['datediff'].dt.days
customer_df['recency_ranking']=pd.qcut(customer_df['datediff'],q=4,labels=[1,2,3,4])
recency=customer_df[['datediff', 'customer_id','recency_ranking']].sort_values(by='datediff', ascending=True)
recency

,datediff,customer_id,recency_ranking
38270,273,C196970,1
54527,273,C276677,1
89046,273,C788712,1
54153,273,C290329,1
10182,273,C224033,1
...,...,...,...
45626,1069,C260427,4
45697,1069,C316581,4
5323,1069,C105769,4
69370,1069,C824542,4


Calculation of Frequency


In [32]:
customer_frequency=customer_df.groupby('customer_id')['invoice_no'].nunique()
customer_df['frequency']=customer_df['customer_id'].map(customer_frequency)
customer_df['ranking']=1
result_of_frequency=customer_df[['customer_id','frequency','ranking']]
result_of_frequency

,customer_id,frequency,ranking
0,C241288,1,1
1,C111565,1,1
2,C266599,1,1
3,C988172,1,1
4,C189076,1,1
...,...,...,...
99452,C441542,1,1
99453,C569580,1,1
99454,C103292,1,1
99455,C800631,1,1


Calculation of Monetary

In [33]:
customer_df['total_spent'] = customer_df['quantity'] * customer_df['price']
customer_total_spending=customer_df.groupby(['shopping_mall','customer_id'])['total_spent'].sum().reset_index()
customer_total_spending['monetary_ranking']=pd.qcut(customer_total_spending['total_spent'], q=4,labels=[1,2,3,4])
customer_total_spending=customer_total_spending.sort_values(by=['total_spent','monetary_ranking'],ascending=False)
customer_total_spending

,shopping_mall,customer_id,total_spent,monetary_ranking
182,Cevahir AVM,C110842,26250,4
471,Cevahir AVM,C131264,26250,4
518,Cevahir AVM,C134372,26250,4
528,Cevahir AVM,C134690,26250,4
599,Cevahir AVM,C138567,26250,4
...,...,...,...,...
99315,Zorlu Center,C929807,5,1
99327,Zorlu Center,C937086,5,1
99332,Zorlu Center,C938777,5,1
99405,Zorlu Center,C967964,5,1


Result

In [41]:
first_merge=customer_df.merge(customer_total_spending,on=['customer_id','total_spent'])
result=recency.merge(first_merge,on='customer_id')
last_result=result[['customer_id','total_spent','recency_ranking_x','frequency','monetary_ranking']]
last_result=last_result.sort_values(by='total_spent',ascending=False)
last_result

,customer_id,total_spent,recency_ranking_x,frequency,monetary_ranking
91183,C118230,26250,4,1,4
8669,C276486,26250,1,1,4
13922,C328698,26250,1,1,4
78536,C472297,26250,4,1,4
93399,C164998,26250,4,1,4
...,...,...,...,...,...
87093,C212687,5,4,1,1
64638,C215013,5,3,1,1
8436,C326569,5,1,1,1
8432,C338868,5,1,1,1


Segmentation

In [35]:
def segment(recency_ranking_x, frequency, monetary_ranking):
    if recency_ranking_x==1 and frequency==1 and monetary_ranking==1:
        return 'Best Customers'
    elif recency_ranking_x==1 and frequency==4 and monetary_ranking==1 or monetary_ranking==0:
        return 'High Spending New Customers'
    elif recency_ranking_x==1 and frequency==1 and monetary_ranking==3 or monetary_ranking==4:
        return 'Lowest Spending'
    elif recency_ranking_x==4 and frequency==1 or frequency==2 and monetary_ranking==1 or monetary_ranking==2:
       return 'Churned Best Customers'
    else: 
        return 'other customers'

result2=result2.sort_values(by='total_spent',ascending=False)
result2['Segment'] = result2.apply(lambda row: segment(row['recency_ranking_x'], row['frequency'], row['monetary_ranking']), axis=1)
result2 = result2.sort_values(by=['Segment'], ascending=True)
result2[['customer_id','recency_ranking_x','frequency','monetary_ranking','Segment']]

,customer_id,recency_ranking_x,frequency,monetary_ranking,Segment
24236,C333928,1,1,1,Best Customers
6020,C521933,1,1,1,Best Customers
11620,C856801,1,1,1,Best Customers
6062,C126206,1,1,1,Best Customers
23107,C109124,1,1,1,Best Customers
...,...,...,...,...,...
57577,C979121,3,1,3,other customers
51811,C852642,3,1,3,other customers
61542,C290521,3,1,3,other customers
26975,C180784,2,1,3,other customers
